In [ ]:
%load_ext blackcellmagic
%alias_magic -c -p "-l 120" b black

# MEI Demo

In [ ]:
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config["schema_name"] = "nnfabrik_tutorial"

schema = dj.schema("nnfabrik_tutorial")

In [ ]:
import os

from matplotlib import pyplot as plt
from torch import load

from featurevis.main import TrainedEnsembleModelTemplate, CSRFV1SelectorTemplate, MEISeed, MEIMethod, MEITemplate
from nnfabrik.template import TrainedModelBase
from nnfabrik.main import Dataset

## 1. Define Tables

In [ ]:
@schema
class TrainedModel(TrainedModelBase):
    pass


@schema
class TrainedEnsembleModel(TrainedEnsembleModelTemplate):
    dataset_table = Dataset
    trained_model_table = TrainedModel


@schema
class CSRFV1Selector(CSRFV1SelectorTemplate):
    dataset_table = Dataset


@schema
class MEI(MEITemplate):
    trained_model_table = TrainedEnsembleModel
    selector_table = CSRFV1Selector

## 2. Reset Tables For Demo

In [ ]:
CSRFV1Selector().drop()
TrainedEnsembleModel().drop()
MEIMethod().drop()
MEISeed().drop()

## 3. Create Ensemble Model

In [ ]:
TrainedEnsembleModel().create_ensemble(dict(dataset_fn="csrf_v1"), comment="Happy little ensemble")
TrainedEnsembleModel()

## 4. Populate Selector Table

In [ ]:
CSRFV1Selector.populate()
CSRFV1Selector()

## 5. Specify MEI Method Parameters

In [ ]:
method_fn = "featurevis.methods.gradient_ascent"
method_config = dict(
    optimizer=dict(path="torch.optim.SGD", kwargs=dict(lr=0.1)),
    stopper=dict(path="featurevis.stoppers.NumIterations", kwargs=dict(num_iterations=1000)),
    objectives=[dict(path="featurevis.objectives.EvaluationObjective", kwargs=dict(interval=10))],
    device="cuda",
)
MEIMethod().add_method(method_fn, method_config)
MEIMethod()

## 6. Add Seed

In [ ]:
MEISeed().insert1(dict(mei_seed=42))
MEISeed()

## 7. Generate MEIs

In [ ]:
MEI().populate(dict(neuron_id=188), display_progress=True)
MEI()

## 8. Look at MEIs

In [ ]:
neuron_id = 188
mei_path = (MEI() & dict(ensemble_id=0, neuron_id=neuron_id)).fetch1("mei")
plt.imshow(load(mei_path).squeeze(), cmap="gray")
plt.gca().axis("off")
os.remove(mei_path)

## 9. Plot Evaluations Across Time

In [ ]:
output_path = (MEI() & dict(ensemble_id=0, neuron_id=neuron_id)).fetch1("output")
output = load(output_path)
os.remove(output_path)

plt.plot(
    output["featurevis.objectives.EvaluationObjective"]["times"],
    output["featurevis.objectives.EvaluationObjective"]["values"],
)
plt.gca().set_xlabel("# iteration")
plt.gca().set_ylabel("evaluation")